## Face recognition

In [1]:
import time
st = time.time()

### Cloning repository

In [2]:
!git clone https://github.com/arrafi-musabbir/facial-recognition.git
!sudo rm -rf /content/sample_data
!unzip -q /content/facial-recognition/images.zip 

Cloning into 'facial-recognition'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 7), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (33/33), done.


### Installing necessary libraries

In [3]:
!pip install numpy scipy dlib firebase_admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Downloading pretained dlib libraries

In [4]:
!wget http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2022-06-23 04:17:07--  http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21428389 (20M)
Saving to: ‘dlib_face_recognition_resnet_model_v1.dat.bz2’

dlib_face_recogniti 100%[===================>]  20.44M  6.27MB/s    in 3.3s    

2022-06-23 04:17:11 (6.27 MB/s) - ‘dlib_face_recognition_resnet_model_v1.dat.bz2’ saved [21428389/21428389]

--2022-06-23 04:17:11--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  1.04MB/s    in 67s     

2022-06-23 04:18:19 (927 KB/s) - ‘shape_predictor_68_face_landmar

In [5]:
!bzip2 -d /content/dlib_face_recognition_resnet_model_v1.dat.bz2
!bzip2 -d /content/shape_predictor_68_face_landmarks.dat.bz2

### Utility funtions

In [6]:
import dlib
import scipy.misc
import numpy as np
import os
# Get Face Detector from dlib
# This allows us to detect faces in images
face_detector = dlib.get_frontal_face_detector()
# Get Pose Predictor from dlib
# This allows us to detect landmark points in faces and understand the pose/angle of the face
shape_predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')
# Get the face recognition model
# This is what gives us the face encodings (numbers that identify the face of a particular person)
face_recognition_model = dlib.face_recognition_model_v1('/content/dlib_face_recognition_resnet_model_v1.dat')
# This is the tolerance for face comparisons
# The lower the number - the stricter the comparison
# To avoid false matches, use lower value
# To avoid false negatives (i.e. faces of the same person doesn't match), use higher value
# 0.5-0.6 works well
TOLERANCE = 0.4

In [7]:
# This function will take an image and return its face encodings using the neural network
import imageio
import cv2
def get_face_encodings(path_to_image):
    # Load image using scipy
    # image = imageio.imread(path_to_image)
    image = cv2.imread(path_to_image)
    # Detect faces using the face detector
    detected_faces = face_detector(image, 1)
    # Get pose/landmarks of those faces
    # Will be used as an input to the function that computes face encodings
    # This allows the neural network to be able to produce similar numbers for faces of the same people, regardless of camera angle and/or face positioning in the image
    shapes_faces = [shape_predictor(image, face) for face in detected_faces]
    # For every face detected, compute the face encodings
    return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

In [8]:
# This function takes a list of known faces
def compare_face_encodings(known_faces, face):
    # Finds the difference between each known face and the given face (that we are comparing)
    # Calculate norm for the differences with each known face
    # Return an array with True/Face values based on whether or not a known face matched with the given face
    # A match occurs when the (norm) difference between a known face and the given face is less than or equal to the TOLERANCE value
    return (np.linalg.norm(known_faces - face, axis=1) <= TOLERANCE)

In [9]:
# This function returns the name of the person whose image matches with the given face (or 'Not Found')
# known_faces is a list of face encodings
# names is a list of the names of people (in the same order as the face encodings - to match the name with an encoding)
# face is the face we are looking for
def find_match(known_faces, names, face):
    # Call compare_face_encodings to get a list of True/False values indicating whether or not there's a match
    matches = compare_face_encodings(known_faces, face)
    # Return the name of the first match
    count = 0
    for match in matches:
        if match:
            return names[count]
        count += 1
    # Return not found if no match found
    return 'Not Found'

### Training face-recognition model

In [10]:
# Get path to all the known images
# Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('images/'))
# Sort in alphabetical order
image_filenames = sorted(image_filenames)
# Get full paths to images
paths_to_images = ['images/' + x for x in image_filenames]
# List of face encodings we have
face_encodings = []
# Loop over images to get the encoding one by one
for path_to_image in paths_to_images:
    # Get face encodings from the image
    face_encodings_in_image = get_face_encodings(path_to_image)
    # Make sure there's exactly one face in the image
    if len(face_encodings_in_image) != 1:
        print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
        exit()
    # Append the face encoding found in that image to the list of face encodings we have
    try:
      face_encodings.append(get_face_encodings(path_to_image)[0])
    except IndexError:
      print("Couldn't find face in ", path_to_image)

### Creating test directory

In [11]:
os.mkdir('test')

### Clear test directory

In [26]:
import shutil
def clear_test():
    dir = '/content/test'
    for files in os.listdir(dir):
        path = os.path.join(dir, files)
        try:
            shutil.rmtree(path)
        except OSError:
            os.remove(path)

### face recognition


In [55]:
def face_recognition():
    # Get path to all the test images
    # Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
    test_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('test/'))
    # Get full paths to test images
    paths_to_test_images = ['test/' + x for x in test_filenames]
    # Get list of names of people by eliminating the .JPG extension from image filenames
    names = [x[:-4] for x in image_filenames]
    # Iterate over test images to find match one by one
    if len(paths_to_test_images)==0:
        print("No images found")
    for path_to_image in paths_to_test_images:
        # Get face encodings from the test image
        face_encodings_in_image = get_face_encodings(path_to_image)
        # Make sure there's exactly one face in the image
        if len(face_encodings_in_image) != 1:
            print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
            exit()
        # Find match for the face encoding found in this test image
        try:
            match = find_match(face_encodings, names, face_encodings_in_image[0])
            # Print the path of test image and the corresponding match
            print(path_to_image, " >>>> recognized >>>>", match.partition("_")[0])
            face = match.partition("_")[0]
        except:
            face = "No face"
            print("No face detected")
    return face

## Firebase

### Connect to database

In [13]:
import firebase_admin
from firebase_admin import credentials
# connecting to firebase with credentials
cred = credentials.Certificate("/content/facial-recognition/digitalkotemanagementsystem-firebase-adminsdk-6yboy-00b2489f3b.json")
firebase_admin.initialize_app(cred,{'databaseURL':'https://digitalkotemanagementsystem-default-rtdb.firebaseio.com/'})
print("Successfully established connection to database")

Successfully established connection to database


In [14]:
from firebase_admin import db
ref = db.reference("/temp_image/1/")
ref1 = db.reference("recognized_face")

### Fetch image from database

In [15]:
import base64
def fetch_image():
    img64 = ref.get()['img64'].encode()
    image_64_decode = base64.b64decode(img64) 
    image_result = open('test/test.jpg', 'wb')
    image_result.write(image_64_decode)

### Recognize face and update database

In [29]:
def update_database():
    clear_test()
    fetch_image()
    try:
      face = face_recognition()
    except:
      face = "No face"
    new = len(ref1.get())
    new = str(new)
    box_ref = ref1.child('1')
    box_ref = ref1.child('{new}'.format(new=new))
    box_ref.update({
        'name': '{face}'.format(face=face)
    })
    print('Face recognized: {face}'.format(face=face))
    print('Database updated accordingly')

In [30]:
print('The program is ready and executable')
et = time.time()
print("Took {:2f} minutes".format((et-st)/60))

The program is ready and executable
Took 32.199899 minutes


### Run every block before this with run-before command from runtime

## Run this block only-- without app

In [59]:
try:
  face_recognition()
except IndexError:
  print("Couldn't find face")

test/Lt Abdullah_19.jpg  >>>> recognized >>>> Lt Abdullah
test/test.jpg  >>>> recognized >>>> Lt Abdullah
test/Lt Salman_27.jpg  >>>> recognized >>>> Lt Salman
test/Capt Tanzil_292.jpg  >>>> recognized >>>> Capt Tanzil
test/Lt Salman.jpg  >>>> recognized >>>> Lt Salman
test/Fo Mehedi_24.jpg  >>>> recognized >>>> Fo Mehedi


## Run this block only-- hooked with app

In [61]:
update_database()

test/test.jpg  >>>> recognized >>>> Lt Abdullah
Face recognized: Lt Abdullah
Database updated accordingly
